In [ ]:
pip install transformers tensorflow scikit-learn googletrans

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.1


In [ ]:
!pip install tensorflow --upgrade
!pip install transformers --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 752.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


**SENTIMENTAL ANALYSIS TAMIL**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels

# Load the training data from two CSV files
file_path_1 = '/content/Tulu_SA_val.csv'  # Path to the first file
file_path_2 = '/content/Tulu_SA_train.csv'  # Path to the second file

df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combine the datasets
df = pd.concat([df1, df2], ignore_index=True)

# Ensure the column names are consistent
df.columns = ['Text', 'Label']  # Adjust if necessary

# Normalize label values (convert to lowercase for consistency)
df['Label'] = df['Label'].str.lower()

# Inspect unique values in the 'Label' column before encoding
print(f"Unique values in 'Label' column before encoding: {df['Label'].unique()}")

# Map labels to numeric values for multi-class classification
label_mapping = {
    'positive': 0,
    'negative': 1,
    'neutral': 2,
    'mixed_feelings': 3,
    'unknown_state': 4  # Include unknown_state
}
df['Label'] = df['Label'].map(label_mapping)

# Drop rows with missing or invalid labels
df = df.dropna(subset=['Label'])
if df.empty:
    raise ValueError("No valid rows remain after label mapping. Check your data and mapping.")

# Check unique labels after mapping
print(f"Unique numeric labels: {df['Label'].unique()}")

# Split the data into features (X) and target labels (y)
X = df['Text'].values
y = df['Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text data using TF-IDF
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train KNN Model
knn_model = KNeighborsClassifier(n_neighbors=5)  # Use 5 neighbors by default
knn_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = knn_model.predict(X_test_tfidf)

# Define target names for all mapped labels
target_names = ['Positive', 'Negative', 'Neutral', 'Mixed Feelings', 'Unknown State']

# Get unique classes present in y_test and y_pred
present_classes = unique_labels(y_test, y_pred)

# Dynamically filter target names based on present classes
filtered_target_names = [target_names[int(i)] for i in present_classes]

# Generate classification report with filtered target names
report = classification_report(y_test, y_pred, target_names=filtered_target_names)
print(report)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

# Load the test data for prediction (without labels)
test_file_path = '/content/Tulu_SA_test_without_label.csv'
test_df = pd.read_csv(test_file_path)

# Tokenize and vectorize test data
X_test_final_tfidf = tfidf.transform(test_df['Text'].values)

# Make predictions on the test data
y_test_final_pred = knn_model.predict(X_test_final_tfidf)

# Map predicted labels to original labels
reverse_label_map = {0: 'positive', 1: 'negative', 2: 'neutral', 3: 'mixed_feelings', 4: 'unknown_state'}
test_df['Label'] = [reverse_label_map[label] for label in y_test_final_pred]

# Save results to a CSV file
output_file_path = '/content/SAT_results_KNN.csv'
test_df.to_csv(output_file_path, index=False)

print(f'Results saved to {output_file_path}')


Unique values in 'Label' column before encoding: ['positive' 'not tulu' 'mixed' 'neutral' 'negative' nan]
Unique numeric labels: [0. 2. 1.]
              precision    recall  f1-score   support

    Positive       0.64      0.63      0.64       856
    Negative       0.29      0.22      0.25       201
     Neutral       0.53      0.57      0.55       692

    accuracy                           0.56      1749
   macro avg       0.49      0.48      0.48      1749
weighted avg       0.56      0.56      0.56      1749

Test Accuracy: 0.5620354488279017
Results saved to /content/SAT_results_KNN.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels

# Load the training data from two CSV files
file_path_1 = '/content/Tulu_SA_val.csv'  # Path to the first file
file_path_2 = '/content/Tulu_SA_train.csv'  # Path to the second file

df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combine the datasets
df = pd.concat([df1, df2], ignore_index=True)

# Ensure the column names are consistent
df.columns = ['Text', 'Label']  # Adjust if necessary

# Normalize label values (convert to lowercase for consistency)
df['Label'] = df['Label'].str.lower()

# Inspect unique values in the 'Label' column before encoding
print(f"Unique values in 'Label' column before encoding: {df['Label'].unique()}")

# Map labels to numeric values for multi-class classification
label_mapping = {
    'positive': 0,
    'negative': 1,
    'neutral': 2,
    'mixed_feelings': 3,
    'unknown_state': 4  # Include unknown_state
}
df['Label'] = df['Label'].map(label_mapping)

# Drop rows with missing or invalid labels
df = df.dropna(subset=['Label'])
if df.empty:
    raise ValueError("No valid rows remain after label mapping. Check your data and mapping.")

# Check unique labels after mapping
print(f"Unique numeric labels: {df['Label'].unique()}")

# Split the data into features (X) and target labels (y)
X = df['Text'].values
y = df['Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text data using TF-IDF
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train SVM Model
svm_model = SVC(kernel='linear', random_state=42)  # Use linear kernel for text classification
svm_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = svm_model.predict(X_test_tfidf)

# Define target names for all mapped labels
target_names = ['Positive', 'Negative', 'Neutral', 'Mixed Feelings', 'Unknown State']

# Get unique classes present in y_test and y_pred
present_classes = unique_labels(y_test, y_pred)

# Dynamically filter target names based on present classes
filtered_target_names = [target_names[int(i)] for i in present_classes]

# Generate classification report with filtered target names
report = classification_report(y_test, y_pred, target_names=filtered_target_names)
print(report)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

# Load the test data for prediction (without labels)
test_file_path = '/content/Tulu_SA_test_without_label.csv'
test_df = pd.read_csv(test_file_path)

# Tokenize and vectorize test data
X_test_final_tfidf = tfidf.transform(test_df['Text'].values)

# Make predictions on the test data
y_test_final_pred = svm_model.predict(X_test_final_tfidf)

# Map predicted labels to original labels
reverse_label_map = {0: 'positive', 1: 'negative', 2: 'neutral', 3: 'mixed_feelings', 4: 'unknown_state'}
test_df['Label'] = [reverse_label_map[label] for label in y_test_final_pred]

# Save results to a CSV file
output_file_path = '/content/SAT_results_SVM.csv'
test_df.to_csv(output_file_path, index=False)

print(f'Results saved to {output_file_path}')


Unique values in 'Label' column before encoding: ['positive' 'not tulu' 'mixed' 'neutral' 'negative' nan]
Unique numeric labels: [0. 2. 1.]
              precision    recall  f1-score   support

    Positive       0.83      0.83      0.83       856
    Negative       0.73      0.41      0.53       201
     Neutral       0.71      0.80      0.75       692

    accuracy                           0.77      1749
   macro avg       0.76      0.68      0.70      1749
weighted avg       0.77      0.77      0.76      1749

Test Accuracy: 0.7701543739279588
Results saved to /content/SAT_results_SVM.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels

# Load the training data from two CSV files
file_path_1 = '/content/Tulu_SA_train.csv'  # Path to the first file
file_path_2 = '/content/Tulu_SA_val.csv'    # Path to the second file

df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combine the datasets
df = pd.concat([df1, df2], ignore_index=True)

# Ensure the column names are consistent
df.columns = ['Text', 'Label']  # Adjust if necessary

# Normalize label values (convert to lowercase for consistency)
df['Label'] = df['Label'].str.lower()

# Inspect unique values in the 'Label' column before encoding
print(f"Unique values in 'Label' column before encoding: {df['Label'].unique()}")

# Map labels to numeric values for multi-class classification
label_mapping = {
    'positive': 0,
    'negative': 1,
    'neutral': 2,
    'mixed_feelings': 3,
    'unknown_state': 4  # Include unknown_state
}
df['Label'] = df['Label'].map(label_mapping)

# Drop rows with missing or invalid labels
df = df.dropna(subset=['Label'])
if df.empty:
    raise ValueError("No valid rows remain after label mapping. Check your data and mapping.")

# Check unique labels after mapping
print(f"Unique numeric labels: {df['Label'].unique()}")

# Split the data into features (X) and target labels (y)
X = df['Text'].values
y = df['Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize text data using TF-IDF
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Train Decision Tree Model
dt_model = DecisionTreeClassifier(random_state=42)  # Default decision tree classifier
dt_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = dt_model.predict(X_test_tfidf)

# Define target names for all mapped labels
target_names = ['Positive', 'Negative', 'Neutral', 'Mixed Feelings', 'Unknown State']

# Get unique classes present in y_test and y_pred
present_classes = unique_labels(y_test, y_pred)

# Dynamically filter target names based on present classes
filtered_target_names = [target_names[int(i)] for i in present_classes]

# Generate classification report with filtered target names
report = classification_report(y_test, y_pred, target_names=filtered_target_names)
print(report)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

# Load the test data for prediction (without labels)
test_file_path = '/content/Tulu_SA_test_without_label.csv'
test_df = pd.read_csv(test_file_path)

# Tokenize and vectorize test data
X_test_final_tfidf = tfidf.transform(test_df['Text'].values)

# Make predictions on the test data
y_test_final_pred = dt_model.predict(X_test_final_tfidf)

# Map predicted labels to original labels
reverse_label_map = {0: 'positive', 1: 'negative', 2: 'neutral', 3: 'mixed_feelings', 4: 'unknown_state'}
test_df['Label'] = [reverse_label_map[label] for label in y_test_final_pred]

# Save results to a CSV file
output_file_path = '/content/SAT_results_DT.csv'
test_df.to_csv(output_file_path, index=False)

print(f'Results saved to {output_file_path}')


Unique values in 'Label' column before encoding: ['not tulu' 'positive' 'neutral' 'mixed' 'negative' nan]
Unique numeric labels: [0. 2. 1.]
              precision    recall  f1-score   support

    Positive       0.79      0.79      0.79       827
    Negative       0.51      0.38      0.44       177
     Neutral       0.72      0.77      0.74       745

    accuracy                           0.74      1749
   macro avg       0.68      0.65      0.66      1749
weighted avg       0.73      0.74      0.74      1749

Test Accuracy: 0.7392795883361921
Results saved to /content/SAT_results_DT.csv


In [ ]:
!pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296225 sha256=1de1d3a44eda7f4ec253982a41dcf2e4318ac58d22b4a379698633ede2bbc153
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext
